In [ ]:
# The following should reload files if we change them on disk
%load_ext autoreload
%aimport econsim.work, econsim.resources, econsim.makeragent, econsim.designer, econsim.producer, econsim.serviceprovider, econsim.utils, econsim.distmakingmodel, econsim.market
%autoreload 1

from pdb import set_trace

In [ ]:
from utils import generate_ranges
from econsim.distmakingmodel import DistMakingModel

range_weights, range_living_cost, range_threshold, range_competition, range_resources = generate_ranges(2)

In [ ]:
graphic_params = {
    "designers": {
        "type": "SliderInt",
        "value": 20,
        "label": "Designers",
        "min": 0,
        "max": 100,
        "step": 1
    },
    "producers": {
        "type": "SliderInt",
        "value": 5,
        "label": "Producers",
        "min": 0,
        "max": 100,
        "step": 1
    },
    "initial_wealth": {
        "type": "SliderInt",
        "value": 10,
        "label": "Initial Wealth",
        "min": 0,
        "max": 100,
        "step": 1
    },
    "price_weight": {
        "type": "SliderFloat",
        "value": min(range_price_weight),
        "label": "Weight of price",
        "min": min(range_price_weight),
        "max": max(range_price_weight),
        "step": 0.1
    },

    "living_cost": {
        "type": "SliderFloat",
        "value": min(range_living_cost),
        "label": "Cost of Living",
        "min": min(range_living_cost),
        "max": max(range_living_cost),
        "step": 0.1
    },
    "threshold": {
        "type": "SliderFloat",
        "value": min(range_threshold),
        "label": "Threshold to buy",
        "min": min(range_threshold),
        "max": max(range_threshold),
        "step": 0.1
    },
    "competition": {
        "type": "SliderInt",
        "value": 1,
        "label": "Min products on sale",
        "min": 0,
        "max": 5,
        "step": 1
    }
}



In [ ]:
from mesa.experimental import JupyterViz
import solara
from matplotlib.figure import Figure

def agent_portrayal(agent):
    portrayal = {"Shape": "circle", "Filled": "true", "r": 0.5}
    agtype = DistMakingModel.agent_type(agent.unique_id)
    if agtype == "Designer":
        shape = "square"
        color = "tab:red"
    elif agtype == "Producer":
        shape = "circle"
        color = "tab:blue"
    else:
        shape = "triangle"
        color = "tab:green"
    size = 20
    # color = "tab:red"
    # if agent.wealth > 0:
    #     size = 50
    #     color = "tab:blue"
    # "edge_color": edge_color,
    # "node_color": node_color,
    return {"size": size, "color": color, "shape": shape, "edge_color": "tab:red"}

def wealth_histogram(model):
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    wealth_vals = [agent.wealth for agent in model.schedule.agents]
    # Note: you have to use Matplotlib's OOP API instead of plt.hist
    # because plt.hist is not thread-safe.
    ax.hist(wealth_vals, bins=10)
    solara.FigureMatplotlib(fig)

def work_histogram(model):
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    wealth_vals = [agent.worked_hours for agent in model.schedule.agents if hasattr(agent,'worked_hours')]
    # Note: you have to use Matplotlib's OOP API instead of plt.hist
    # because plt.hist is not thread-safe.
    ax.hist(wealth_vals, bins=10)
    solara.FigureMatplotlib(fig)
    
page = JupyterViz(
    DistMakingModel,
    graphic_params,
    measures=["Gini", 
              "Designs in Progress",
              "Realized Designs",
              "Products in Progress",
              "On-sale Products",
              "Sold Products",
              wealth_histogram,
              work_histogram],
    name="Fair Distribution",
    agent_portrayal=agent_portrayal
)
# This is required to render the visualization in the Jupyter notebook
page